### Data Exploration

The research paper uses sentence embedding on noun and noun phrases. This analysis is to see of other modern approaches can reach same or better score. Ultimately, we want to see if the new approach at least mataches the high scores annotated. 

1. Compare embedding of entire resume to individual entities of a category. Expect sim,ilarity to be less than the research paper
2. Colbert index and search. 

In [4]:
import pandas as pd
import json
import os

In [8]:
data_dir = "../data/o_net_files/"

### Run the following code for O*Net Knowledge, Skill, Abilities and Work Activities (KSA&W) Excels to CSV




In [11]:
import os
import pandas as pd

def process_onet_excel_to_csv(filename, data_dir):
    """
    Processes an O*NET Excel file by selecting and renaming specific columns,
    then saves it as a CSV in the same directory with the same base filename.

    The 'Element Name' column will be renamed based on the file name:
    - 'Knowledge.xlsx' → 'knowledge_entity'
    - 'Skills.xlsx' → 'skills_entity'
    - etc.
    """
    full_path = os.path.join(data_dir, filename)
    df_onet = pd.read_excel(full_path)

    # Select relevant columns
    df_onet = df_onet[[
        "O*NET-SOC Code", "Title", "Element Name", 
        "Scale Name", "Scale ID", "Data Value"
    ]]

    # Infer the entity type from the filename
    base_name = os.path.splitext(filename)[0].lower()  # e.g., "knowledge"
    entity_col_name = f"{base_name}_entity"

    # Rename columns
    df_onet.rename(columns={
        "O*NET-SOC Code": "onetsoc_code",
        "Title": "job_title",
        "Element Name": entity_col_name,
        "Scale ID": "scale_id",
        "Scale Name": "scale_name",
        "Data Value": "data_value"
    }, inplace=True)

    # Save CSV
    output_path = os.path.join(data_dir, f"{base_name}.csv")
    df_onet.to_csv(output_path, index=False)
    print(f"✅ Processed and saved: {output_path}")


In [13]:
#  Process all Excel files in a directory
for file in os.listdir(data_dir):
    if file.endswith(".xlsx"):
        process_onet_excel_to_csv(file, data_dir)


✅ Processed and saved: ../data/o_net_files/knowledge.csv
✅ Processed and saved: ../data/o_net_files/skills.csv
✅ Processed and saved: ../data/o_net_files/work_activities.csv
✅ Processed and saved: ../data/o_net_files/abilities.csv


### Run the following to convert O*Net Occupation Excel to CSV

This is optional - in case we want to see relationship with the occupation data

In [ ]:
# # Load the O*NET Occupation Excel file
# occupation_file = "data/annotations_scenario_1/Occupation Data.xlsx"  # Update with the actual filename

# df_occupation = pd.read_excel(occupation_file)

# # Select relevant columns
# df_occupation = df_occupation[["O*NET-SOC Code", "Title", "Description"]]

# # Rename columns for consistency
# df_occupation.rename(columns={
#     "O*NET-SOC Code": "onetsoc_code",
#     "Title": "job_title",
#     "Description": "job_description"
# }, inplace=True)

# # Display the first few rows
# print(df_occupation.head())

# # Save to CSV for further inspection (optional)
# df_occupation.to_csv("data/annotations_scenario_1/processed_onet_occupation.csv", index=False)
